In [1]:
import numpy as np
import pandas as pd 
import string
import re

In [2]:
with open('all_inovices.txt', mode='r', encoding='utf8', errors='ignore') as f:
    text = f.read()

In [4]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [7]:
df = pd.DataFrame(data[1:], columns=data[0])

In [8]:
df.head()

,id,text,,
0,001.jpeg,Invoice,O,
1,001.jpeg,no:,O,
2,001.jpeg,51109338,B-ID,
3,001.jpeg,Date,O,
4,001.jpeg,of,O,
